In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from dateutil import relativedelta

# scomsDF = pd.read_excel('Hack DataSet/scoms_ticket.xlsx')
# activeDF = pd.read_excel('Hack DataSet/active_profile.xlsx')
# inactiveDF = pd.read_excel('Hack DataSet/inactive_profile.xlsx')
# paymentDF = pd.read_excel('Hack DataSet/payment.xlsx')
# billingDF = pd.read_excel('Hack DataSet/billing.xlsx')
import warnings
warnings.filterwarnings('ignore')

In [2]:
# โหลดข้อมูล Scoms
scoms2021_22_df = pd.read_csv('DataSet/FTTx 2021-2022.txt', encoding="ISO-8859-11", on_bad_lines="skip")

In [3]:
scoms2023_df = pd.read_csv('DataSet/FTTx 2023.txt', encoding="ISO-8859-11", on_bad_lines="skip")

In [207]:
# โหลดข้อมูล Custermer profile (Active + Inactive)
inac_active_2023_df = pd.read_csv('DataSet/Customer Active_Inactive.csv')

In [208]:
print("คอลัมน์ที่มีใน scoms2021_22_df แต่ไม่มีใน scoms2023_df:")
print(set(scoms2021_22_df.columns) - set(scoms2023_df.columns))

print("คอลัมน์ที่มีใน scoms2021_22_df แต่ไม่มีใน scoms2023_df:")
print(set(scoms2021_22_df.columns) - set(scoms2023_df.columns))

print("คอลัมน์ที่มีใน scoms2021_22_df และ scoms2023_df:")
same_col = pd.DataFrame(set(scoms2021_22_df.columns) & set(scoms2023_df.columns))
same_col

print("คอลัมน์ที่ไม่มีใน scoms2021_22_df และ scoms2023_df:")
same_col = pd.DataFrame(set(scoms2021_22_df.columns) ^ set(scoms2023_df.columns))
same_col

คอลัมน์ที่มีใน scoms2021_22_df แต่ไม่มีใน scoms2023_df:
set()
คอลัมน์ที่มีใน scoms2021_22_df แต่ไม่มีใน scoms2023_df:
set()
คอลัมน์ที่มีใน scoms2021_22_df และ scoms2023_df:
คอลัมน์ที่ไม่มีใน scoms2021_22_df และ scoms2023_df:


""


### Cleansing Data File ###

In [4]:
#  รวมไฟล์ข้อมูล Scoms
scoms2021_23_df = pd.concat([scoms2021_22_df, scoms2023_df],axis=0)
scoms2021_23_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7188224 entries, 0 to 2281516
Data columns (total 26 columns):
 #   Column          Dtype  
---  ------          -----  
 0   ticket          int64  
 1   number          object 
 2   servicetype     object 
 3   faildate        object 
 4   fcode           object 
 5   fcode_name      object 
 6   testdate        object 
 7   ftest           object 
 8   ftest_name      object 
 9   testflag        int64  
 10  meetwhen        object 
 11  meetdate        object 
 12  comdate         object 
 13  fncode          object 
 14  fncode_name     object 
 15  subgroup        float64
 16  subgroup_name   object 
 17  maingroup       float64
 18  maingroup_name  object 
 19  callback        object 
 20  TagByCus        int64  
 21  TagByEmp        float64
 22  isProactive     float64
 23  wocode          object 
 24  name            object 
 25  Sector          object 
dtypes: float64(4), int64(3), object(19)
memory usage: 1.4+ GB


In [5]:
scoms2023_df.count()

ticket            2281517
number            2281517
servicetype       2255068
faildate          2281517
fcode             2281517
fcode_name        2280043
testdate           560191
ftest              370821
ftest_name         370821
testflag          2281517
meetwhen          2281517
meetdate          2281517
comdate           1915237
fncode            2281497
fncode_name       1754951
subgroup          1474617
subgroup_name     1474617
maingroup         1474672
maingroup_name    1474672
callback          2281517
TagByCus          2281517
TagByEmp          2060452
isProactive        527006
wocode            2281516
name              2280189
Sector            2280189
dtype: int64

In [210]:
# Drop unnescesary columns
scoms2021_23_df =  scoms2021_23_df.drop(columns=['ticket', 'servicetype', 'faildate', 'fcode', 'fcode_name','testdate', 'ftest', 'ftest_name', 'testflag', 'meetwhen', 'meetdate', 'comdate', 'fncode', 'fncode_name', 'subgroup', 'subgroup_name', 'maingroup', 'maingroup_name', 'callback', 'TagByCus', 'TagByEmp','isProactive', 'wocode', 'name', 'Sector'])
scoms2021_23_df.columns

Index(['number'], dtype='object')

In [211]:
# แสดงเลขหมายที่มีการแจ้งเหตุมากกว่า 1 ครั้ง (ทุก record)
ids = scoms2021_23_df["number"]
# new_scomsDF[ids.isin(ids[ids.duplicated()])].sort_values("number2")
scoms2021_23_df[ids.isin(ids[ids.duplicated()])].sort_values("number").set_index('number').to_csv('DataSet/scoms_dup_record2023.csv',encoding="ISO-8859-11")

In [212]:
# นับจำนวนครั้งที่มีการแจ้งเหตุของแต่ละเลขหมาย พร้อมบันทึกลงไฟล์
count_group = pd.DataFrame()
# count_group = new_scomsDF.groupby(['number2'])['number2'].unique()
# count_group = new_scomsDF.groupby(['number2'])['number2'].value_counts()
count_group = scoms2021_23_df.groupby(['number'])['number'].count()
count_dup_df = count_group.to_frame(name='claims')
count_dup_df = count_dup_df.reset_index()
# count_df
count_dup_df = count_dup_df.rename(columns={'number': 'Number'})
count_dup_df.set_index('Number').to_csv('DataSet/scoms-dup-record-group2023.csv',encoding='utf-8')
count_dup_df

,Number,claims
0,2131J0110,2
1,2131J5032,3
2,2131J5104,9
3,2131J5162,1
4,2131J5184,5
...,...,...
1853153,7789j0123,6
1853154,7789j0128,7
1853155,7789j0130,1
1853156,7789j0131,2


2. Merge Data Customer & Scoms

In [325]:
count_dup_df['claims'].sort_values().value_counts(ascending=False)

claims
1      658842
2      357442
3      227104
4      152936
5      106495
        ...  
289         1
288         1
284         1
283         1
460         1
Name: count, Length: 331, dtype: int64

In [330]:
cd = count_dup_df[count_dup_df['claims']==460]

In [327]:
cd

0          False
1          False
2          False
3          False
4          False
           ...  
1853153    False
1853154    False
1853155    False
1853156    False
1853157    False
Name: claims, Length: 1853158, dtype: bool

In [213]:
# Join ไพล์ข้อมูลลูกค้า กับเลขหมายที่แจ้งเหตุขัดข้อง (active, inactive, scoms)
leftJoin_ac_in_scoms_df = pd.merge(inac_active_2023_df,count_dup_df,left_on='New_name',right_on='Number',how='left')
# leftJoin_ac_in_scoms.set_index('หมายเลขวงจร').to_csv('Hack DataSet/NT Insight DataSet/join_active_inactive_scoms.csv',encoding='utf-8')

In [ ]:
leftJoin_ac_in_scoms_df.info()

In [214]:
# เปลี่ยน Claim = NaN -> 0
leftJoin_ac_in_scoms_df['claims'] = leftJoin_ac_in_scoms_df['claims'].fillna(0)
leftJoin_ac_in_scoms_df['claims'] = leftJoin_ac_in_scoms_df['claims'].astype(int)

In [ ]:
# leftJoin_ac_in_scoms = leftJoin_ac_in_scoms[leftJoin_ac_in_scoms['โปรโมชั่น'] != 'Smart City Paklok+CCTV']

In [ ]:
leftJoin_ac_in_scoms_df.head(2)

In [ ]:
leftJoin_ac_in_scoms_df.columns

In [215]:
leftJoin_ac_in_scoms_df = leftJoin_ac_in_scoms_df.drop(columns=['disconnect_reason',
       'product', 'status','service_location_code', 'service_location_name', 'Location_name_disp','Type', 'ORG05', 'ORG05_ABBR', 'ORG04', 'ORG04_ABBR', 'ORG03', 'ORG03_ABBR', 'Number',])

In [ ]:
leftJoin_ac_in_scoms_df.columns

In [ ]:
leftJoin_ac_in_scoms_df.head(2)

In [ ]:
# leftJoin_ac_in_scoms = leftJoin_ac_in_scoms.drop(columns=['แพ็คเก็จ','Number'])

In [216]:
leftJoin_ac_in_scoms_df['ba_id'] = leftJoin_ac_in_scoms_df['ba_id'].astype(object)

In [217]:
# คำนวนหาระยะเวลาการเป็นลูกค้า
# กำหนดรูปแบบวันที่และเวลา
# date_format = '%Y-%m-%d %H:%M:%S'
date_format = '%Y-%m-%d'

# แปลงข้อมูลในคอลัมน์ start_date และ end_date เป็นวันที่
leftJoin_ac_in_scoms_df['install_date'] = pd.to_datetime(leftJoin_ac_in_scoms_df['install_date'], format=date_format)
leftJoin_ac_in_scoms_df['inactive_date'] = pd.to_datetime(leftJoin_ac_in_scoms_df['inactive_date'], format=date_format, errors='coerce')


In [228]:

# คำนวณความแตกต่างของเวลา และแปลงเป็นเดือน
today = datetime.now().date()

# Calculate Tenure-Month
leftJoin_ac_in_scoms_df['tenure_month'] = (pd.Timestamp(today) - pd.to_datetime(leftJoin_ac_in_scoms_df['install_date']))/ pd.Timedelta(days=30.436875)

leftJoin_ac_in_scoms_df['tenure_month'] = leftJoin_ac_in_scoms_df['tenure_month'].apply(lambda x: round(x))

In [229]:
leftJoin_ac_in_scoms_df['tenure_month']

0         81
1         31
2         80
3         40
4         63
          ..
159995     9
159996    11
159997     6
159998    17
159999    68
Name: tenure_month, Length: 160000, dtype: int64

In [230]:
leftJoin_ac_in_scoms_df.rename(columns={'Active/Inactive':'active_status'},inplace=True)

In [232]:
leftJoin_ac_in_scoms_df.rename(columns={'New_name':'circuit_id'},inplace=True)

In [234]:
leftJoin_ac_in_scoms_df.rename(columns={'ORG_DEP_ABBR':'org_dep_abbr'},inplace=True)


In [235]:
leftJoin_ac_in_scoms_df.columns

Index(['circuit_id', 'install_date', 'inactive_date', 'package',
       'package_startdate', 'package_enddate', 'ba_id', 'customer_type_name',
       'acc_type_update', 'org_dep_abbr', 'package_group_name_th',
       'package_info_name', 'start_date_sale_package', 'end_date_sale_package',
       'package_price', 'active_status', 'claims', 'tenure_month'],
      dtype='object')

In [ ]:
leftJoin_ac_in_scoms_df.head(2)

In [195]:
leftJoin_ac_in_scoms_df['package'].value_counts()

package
ค่าบริการ Fiber 2U 100Mb/100Mb_Gigabolt Max For DE                                                                              16527
ค่าบริการ Fiber 2U 300Mb/300Mb_Gigabolt Max For DE                                                                              13742
ค่าบริการ NT ValueMAX Fiber 300Mb/300Mb (จ่ายพร้อมใช้)                                                                           7358
ค่าบริการ Fiber 2U 1Gb/200Mb_Gigabolt Max                                                                                        6986
ค่าบริการ NT MAX DFiber 300Mbps/300Mbps (จ่ายพร้อมใช้)                                                                           6600
                                                                                                                                ...  
PM706 : ค่าบริการ SMART Home 400Mb/400Mb + Mesh WiFi                                                                                1
PM548 : ค่าบริการ Fiber 2U 250Mb/250Mb เน็ตทั่วบ้าน   

In [236]:
#  Split column โปรโมชั่นเพื่อแยก bundled service: iphone,mobile,tv_box,จ่ายพร้อมใช้,ใช้ก่อนจ่าย

# สร้างฟังก์ชันสำหรับตรวจสอบเงื่อนไข
def check_condition(value, condition):
    if condition in value:
        return 1
    else:
        return 0

# สร้าง column ใหม่สำหรับเงื่อนไขต่างๆ
leftJoin_ac_in_scoms_df['ipphone'] = leftJoin_ac_in_scoms_df['package'].apply(lambda x: check_condition(x, 'hone'))
leftJoin_ac_in_scoms_df['tv_box'] = leftJoin_ac_in_scoms_df['package'].apply(lambda x: check_condition(x, 'C nema') or check_condition(x, 'IPTV'))
leftJoin_ac_in_scoms_df['WiFi'] = leftJoin_ac_in_scoms_df['package'].apply(lambda x: check_condition(x, 'WiFi'))
leftJoin_ac_in_scoms_df['pay_first'] = leftJoin_ac_in_scoms_df['package'].apply(lambda x: check_condition(x, 'จ่ายพร้อมใช้'))
leftJoin_ac_in_scoms_df['mobile'] = leftJoin_ac_in_scoms_df['package'].apply(lambda x: check_condition(x, 'mobile') or check_condition(x, 'SIM'))
leftJoin_ac_in_scoms_df['active_status'] = leftJoin_ac_in_scoms_df['active_status'].apply(lambda x: check_condition(x,'Active'))

# แสดงผลลัพธ์
leftJoin_ac_in_scoms_df.head(2)

# data['สถานะบริการ'].apply(lambda x: 1 if x == 'ยกเลิกถาวร' else 0)Pay later

,circuit_id,install_date,inactive_date,package,package_startdate,package_enddate,ba_id,customer_type_name,acc_type_update,org_dep_abbr,...,end_date_sale_package,package_price,active_status,claims,tenure_month,ipphone,tv_box,WiFi,pay_first,mobile
0,4221J1183,2017-08-23,2022-01-14,ค่าบริการ Fiber 2U 300Mb/300Mb_Gigabolt Max Fo...,2021-04-20,NaN,104530827051,ลูกค้าบ้านพักอาศัยประเภททั่วไป,R,บภน.2.2 (อด.),...,31 ธ.ค. 2022,360.0,0,0,81,0,0,0,0,0
1,7351J4969,2021-11-05,2022-05-11,ค่าบริการ Fiber 2U 300Mb/300Mb_Gigabolt Max Fo...,2021-11-05,NaN,104533018283,ลูกค้าบ้านพักอาศัยประเภททั่วไป,R,บภก.2.1 (นธ.),...,31 ธ.ค. 2022,360.0,0,0,31,0,0,0,0,0


In [ ]:
leftJoin_ac_in_scoms_df.info()

In [237]:
leftJoin_ac_in_scoms_df['package_price'].notna().value_counts()

package_price
True     159978
False        22
Name: count, dtype: int64

In [238]:
leftJoin_ac_in_scoms_df = leftJoin_ac_in_scoms_df[leftJoin_ac_in_scoms_df['package_price'].notna()]
leftJoin_ac_in_scoms_df

,circuit_id,install_date,inactive_date,package,package_startdate,package_enddate,ba_id,customer_type_name,acc_type_update,org_dep_abbr,...,end_date_sale_package,package_price,active_status,claims,tenure_month,ipphone,tv_box,WiFi,pay_first,mobile
0,4221J1183,2017-08-23,2022-01-14,ค่าบริการ Fiber 2U 300Mb/300Mb_Gigabolt Max Fo...,2021-04-20,NaN,104530827051,ลูกค้าบ้านพักอาศัยประเภททั่วไป,R,บภน.2.2 (อด.),...,31 ธ.ค. 2022,360.0,0,0,81,0,0,0,0,0
1,7351J4969,2021-11-05,2022-05-11,ค่าบริการ Fiber 2U 300Mb/300Mb_Gigabolt Max Fo...,2021-11-05,NaN,104533018283,ลูกค้าบ้านพักอาศัยประเภททั่วไป,R,บภก.2.1 (นธ.),...,31 ธ.ค. 2022,360.0,0,0,31,0,0,0,0,0
2,4481J6998,2017-10-04,2022-09-16,ค่าบริการ Fiber 2U 1Gb/500Mb_Gigabolt Max,2021-03-27,NaN,104530876061,ลูกค้าบ้านพักอาศัยประเภททั่วไป,R,บภน.1.2 (ชภ.),...,31 ธ.ค. 2022,700.0,0,0,80,0,0,0,0,0
3,7435J8492,2021-01-18,2022-07-19,ค่าบริการ Fiber 2U 300Mb/300Mb_Gigabolt Max,2021-01-18,NaN,104532765095,ลูกค้าบ้านพักอาศัยประเภททั่วไป,R,บภก.2.1 (สข.),...,31 ธ.ค. 2022,390.0,0,0,40,0,0,0,0,0
4,3223J8807,2019-03-10,2022-04-22,ค่าบริการ Fiber 2U 500Mb/500Mb_Gigabolt Max,2020-12-17,NaN,114404246,ลูกค้าบ้านพักอาศัยประเภททั่วไป,R,บภก.1.2 (รบ.),...,31 ธ.ค. 2022,490.0,0,0,63,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159995,5624J1894,2023-08-30,NaT,ค่าบริการ NT DES Fiber Package 300Mbps/300Mbps,2023-11-20,NaN,104533363342,ลูกค้าบ้านพักอาศัยประเภททั่วไป,R,บภก.1.1 (นว.),...,31 ส.ค. 2024,260.0,1,0,9,0,0,0,0,0
159996,7320J4879,2023-06-26,NaT,ค่าบริการ NT Super Plus Fiber 300Mbps/300Mbps ...,2023-06-26,NaN,104533327498,ลูกค้าบ้านพักอาศัยประเภททั่วไป,R,บภก.3.2 (ยล.),...,30 เม.ย. 2024,290.0,1,0,11,0,0,0,1,0
159997,5558J9574,2023-11-15,NaT,PM805 : ค่าบริการ Fiber 2U 1000Mbps/500Mbps (จ...,2023-11-15,NaN,104529758183,ลูกค้าบ้านพักอาศัยประเภททั่วไป,R,บภน.1.2 (ตก.),...,31 ธ.ค. 2023,295.0,1,0,6,0,0,0,1,0
159998,5356J6470,2022-12-21,NaT,ค่าบริการ NT Super Plus Fiber 600Mbps/600Mbps ...,2023-12-06,NaN,104533246907,ลูกค้าบ้านพักอาศัยประเภททั่วไป,R,บภน.1.1 (ลพ.),...,30 เม.ย. 2024,490.0,1,0,17,0,0,0,1,0


In [239]:
leftJoin_ac_in_scoms_df['acc_type_update'].value_counts()

acc_type_update
R    159978
Name: count, dtype: int64

In [240]:
leftJoin_ac_in_scoms_df['package_price'].isna().value_counts()

package_price
False    159978
Name: count, dtype: int64

In [241]:
# จัดเรียงค่าตามราคาจากมากไปน้อย
df = leftJoin_ac_in_scoms_df.sort_values(by='package_price', ascending=False)

# แสดงราคาสูงสุด 10 อันดับ
df['package_price'].head(10)
# print(top_5_prices)


53242     38070.0
12193      6900.0
3354       6900.0
118115     6900.0
125774     6150.0
83620      4400.0
18507      3890.0
12016      3690.0
120088     3590.0
65507      3590.0
Name: package_price, dtype: float64

In [242]:
# leftJoin_ac_in_scoms = leftJoin_ac_in_scoms[leftJoin_ac_in_scoms['Pay-per-Month'] != leftJoin_ac_in_scoms['Pay-per-Month']>=5890]
leftJoin_ac_in_scoms_df = leftJoin_ac_in_scoms_df[leftJoin_ac_in_scoms_df['package_price'] <=6900]
leftJoin_ac_in_scoms_df = leftJoin_ac_in_scoms_df[leftJoin_ac_in_scoms_df['package_price'] >=100]
# leftJoin_ac_in_scoms_df['package_price'].min()

In [245]:
leftJoin_ac_in_scoms_df['tenure_month'].sort_values(ascending=False)

53067     168
36402     157
127569    153
23838     153
112402    153
         ... 
143356      4
99803       4
151657      4
151659      4
109823      4
Name: tenure_month, Length: 159976, dtype: int64

In [246]:
# ตรวจสอบค่าว่าง
print(leftJoin_ac_in_scoms_df.columns[leftJoin_ac_in_scoms_df.isna().any()])

Index(['inactive_date', 'package_enddate', 'customer_type_name',
       'org_dep_abbr', 'start_date_sale_package', 'end_date_sale_package'],
      dtype='object')


In [253]:
import re
# Create new columns for dl_speed and ul_speed
leftJoin_ac_in_scoms_df['dl_speed'] = None
leftJoin_ac_in_scoms_df['ul_speed'] = None

# Define regular expressions for speed extraction
speed_regex = r"(\d+)(Mbps|Mb|Gb)/(\d+)(Mbps|Mb|Gb)"

# Populate dl_speed and ul_speed columns based on Service-Name
for index, row in leftJoin_ac_in_scoms_df.iterrows():
    service_name = row['package']
    
    match = re.search(speed_regex, service_name)
    if match:
        dl_speed_value = int(match.group(1))
        dl_speed_unit = match.group(2)
        ul_speed_value = int(match.group(3))
        ul_speed_unit = match.group(4)
        
        if dl_speed_unit == 'Gb':
            dl_speed_value *= 1000  # Convert Gb to Mb
        if ul_speed_unit == 'Gb':
            ul_speed_value *= 1000  # Convert Gb to Mb
        
        leftJoin_ac_in_scoms_df.at[index, 'dl_speed'] = dl_speed_value
        leftJoin_ac_in_scoms_df.at[index, 'ul_speed'] = ul_speed_value



In [254]:
leftJoin_ac_in_scoms_df.columns

Index(['circuit_id', 'install_date', 'inactive_date', 'package',
       'package_startdate', 'package_enddate', 'ba_id', 'customer_type_name',
       'acc_type_update', 'org_dep_abbr', 'package_group_name_th',
       'package_info_name', 'start_date_sale_package', 'end_date_sale_package',
       'package_price', 'active_status', 'claims', 'tenure_month', 'ipphone',
       'tv_box', 'WiFi', 'pay_first', 'mobile', 'dl_speed', 'ul_speed'],
      dtype='object')

In [255]:
leftJoin_ac_in_scoms_df

,circuit_id,install_date,inactive_date,package,package_startdate,package_enddate,ba_id,customer_type_name,acc_type_update,org_dep_abbr,...,active_status,claims,tenure_month,ipphone,tv_box,WiFi,pay_first,mobile,dl_speed,ul_speed
0,4221J1183,2017-08-23,2022-01-14,ค่าบริการ Fiber 2U 300Mb/300Mb_Gigabolt Max Fo...,2021-04-20,NaN,104530827051,ลูกค้าบ้านพักอาศัยประเภททั่วไป,R,บภน.2.2 (อด.),...,0,0,81,0,0,0,0,0,300,300
1,7351J4969,2021-11-05,2022-05-11,ค่าบริการ Fiber 2U 300Mb/300Mb_Gigabolt Max Fo...,2021-11-05,NaN,104533018283,ลูกค้าบ้านพักอาศัยประเภททั่วไป,R,บภก.2.1 (นธ.),...,0,0,31,0,0,0,0,0,300,300
2,4481J6998,2017-10-04,2022-09-16,ค่าบริการ Fiber 2U 1Gb/500Mb_Gigabolt Max,2021-03-27,NaN,104530876061,ลูกค้าบ้านพักอาศัยประเภททั่วไป,R,บภน.1.2 (ชภ.),...,0,0,80,0,0,0,0,0,1000,500
3,7435J8492,2021-01-18,2022-07-19,ค่าบริการ Fiber 2U 300Mb/300Mb_Gigabolt Max,2021-01-18,NaN,104532765095,ลูกค้าบ้านพักอาศัยประเภททั่วไป,R,บภก.2.1 (สข.),...,0,0,40,0,0,0,0,0,300,300
4,3223J8807,2019-03-10,2022-04-22,ค่าบริการ Fiber 2U 500Mb/500Mb_Gigabolt Max,2020-12-17,NaN,114404246,ลูกค้าบ้านพักอาศัยประเภททั่วไป,R,บภก.1.2 (รบ.),...,0,0,63,0,0,0,0,0,500,500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159995,5624J1894,2023-08-30,NaT,ค่าบริการ NT DES Fiber Package 300Mbps/300Mbps,2023-11-20,NaN,104533363342,ลูกค้าบ้านพักอาศัยประเภททั่วไป,R,บภก.1.1 (นว.),...,1,0,9,0,0,0,0,0,300,300
159996,7320J4879,2023-06-26,NaT,ค่าบริการ NT Super Plus Fiber 300Mbps/300Mbps ...,2023-06-26,NaN,104533327498,ลูกค้าบ้านพักอาศัยประเภททั่วไป,R,บภก.3.2 (ยล.),...,1,0,11,0,0,0,1,0,300,300
159997,5558J9574,2023-11-15,NaT,PM805 : ค่าบริการ Fiber 2U 1000Mbps/500Mbps (จ...,2023-11-15,NaN,104529758183,ลูกค้าบ้านพักอาศัยประเภททั่วไป,R,บภน.1.2 (ตก.),...,1,0,6,0,0,0,1,0,1000,500
159998,5356J6470,2022-12-21,NaT,ค่าบริการ NT Super Plus Fiber 600Mbps/600Mbps ...,2023-12-06,NaN,104533246907,ลูกค้าบ้านพักอาศัยประเภททั่วไป,R,บภน.1.1 (ลพ.),...,1,0,17,0,0,0,1,0,600,600


In [292]:
leftJoin_ac_in_scoms_df.drop_duplicates(subset='circuit_id', keep='first', inplace=True)

In [293]:
leftJoin_ac_in_scoms_df

,circuit_id,install_date,inactive_date,package,package_startdate,package_enddate,ba_id,customer_type_name,acc_type_update,org_dep_abbr,...,active_status,claims,tenure_month,ipphone,tv_box,WiFi,pay_first,mobile,dl_speed,ul_speed
0,4221J1183,2017-08-23,2022-01-14,ค่าบริการ Fiber 2U 300Mb/300Mb_Gigabolt Max Fo...,2021-04-20,NaN,104530827051,ลูกค้าบ้านพักอาศัยประเภททั่วไป,R,บภน.2.2 (อด.),...,0,0,81,0,0,0,0,0,300,300
1,7351J4969,2021-11-05,2022-05-11,ค่าบริการ Fiber 2U 300Mb/300Mb_Gigabolt Max Fo...,2021-11-05,NaN,104533018283,ลูกค้าบ้านพักอาศัยประเภททั่วไป,R,บภก.2.1 (นธ.),...,0,0,31,0,0,0,0,0,300,300
2,4481J6998,2017-10-04,2022-09-16,ค่าบริการ Fiber 2U 1Gb/500Mb_Gigabolt Max,2021-03-27,NaN,104530876061,ลูกค้าบ้านพักอาศัยประเภททั่วไป,R,บภน.1.2 (ชภ.),...,0,0,80,0,0,0,0,0,1000,500
3,7435J8492,2021-01-18,2022-07-19,ค่าบริการ Fiber 2U 300Mb/300Mb_Gigabolt Max,2021-01-18,NaN,104532765095,ลูกค้าบ้านพักอาศัยประเภททั่วไป,R,บภก.2.1 (สข.),...,0,0,40,0,0,0,0,0,300,300
4,3223J8807,2019-03-10,2022-04-22,ค่าบริการ Fiber 2U 500Mb/500Mb_Gigabolt Max,2020-12-17,NaN,114404246,ลูกค้าบ้านพักอาศัยประเภททั่วไป,R,บภก.1.2 (รบ.),...,0,0,63,0,0,0,0,0,500,500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159995,5624J1894,2023-08-30,NaT,ค่าบริการ NT DES Fiber Package 300Mbps/300Mbps,2023-11-20,NaN,104533363342,ลูกค้าบ้านพักอาศัยประเภททั่วไป,R,บภก.1.1 (นว.),...,1,0,9,0,0,0,0,0,300,300
159996,7320J4879,2023-06-26,NaT,ค่าบริการ NT Super Plus Fiber 300Mbps/300Mbps ...,2023-06-26,NaN,104533327498,ลูกค้าบ้านพักอาศัยประเภททั่วไป,R,บภก.3.2 (ยล.),...,1,0,11,0,0,0,1,0,300,300
159997,5558J9574,2023-11-15,NaT,PM805 : ค่าบริการ Fiber 2U 1000Mbps/500Mbps (จ...,2023-11-15,NaN,104529758183,ลูกค้าบ้านพักอาศัยประเภททั่วไป,R,บภน.1.2 (ตก.),...,1,0,6,0,0,0,1,0,1000,500
159998,5356J6470,2022-12-21,NaT,ค่าบริการ NT Super Plus Fiber 600Mbps/600Mbps ...,2023-12-06,NaN,104533246907,ลูกค้าบ้านพักอาศัยประเภททั่วไป,R,บภน.1.1 (ลพ.),...,1,0,17,0,0,0,1,0,600,600


In [303]:
leftJoin_ac_in_scoms_df = leftJoin_ac_in_scoms_df[leftJoin_ac_in_scoms_df['dl_speed'].notna()]

In [306]:
leftJoin_ac_in_scoms_df['ul_speed'].isna().sum()

0

In [307]:
# แสดงผลลัพธ์
leftJoin_ac_in_scoms_df.set_index('circuit_id').to_csv('DataSet/active_inactive_scoms.csv',encoding='utf-8')
leftJoin_ac_in_scoms_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 159053 entries, 0 to 159999
Data columns (total 25 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   circuit_id               159053 non-null  object        
 1   install_date             159053 non-null  datetime64[ns]
 2   inactive_date            79540 non-null   datetime64[ns]
 3   package                  159053 non-null  object        
 4   package_startdate        159053 non-null  object        
 5   package_enddate          68 non-null      object        
 6   ba_id                    159053 non-null  object        
 7   customer_type_name       158314 non-null  object        
 8   acc_type_update          159053 non-null  object        
 9   org_dep_abbr             158918 non-null  object        
 10  package_group_name_th    159053 non-null  object        
 11  package_info_name        159053 non-null  object        
 12  start_date_sale_packa

In [ ]:
# leftJoin_ac_in_scoms.to_excel('Hack DataSet/join_active_inactive_scoms.xlsx')

In [295]:
# ตรวจสอบ row โดยเงื่อนไขต่างๆ
conditions1 = [
   (leftJoin_ac_in_scoms_df.duplicated(subset=['circuit_id', 'install_date'], keep=False)),
   (leftJoin_ac_in_scoms_df['inactive_date'].isna())
]
# conditions2 = [
#    (leftJoin_ac_in_scoms['Service-Name']=='ipphoneonly'),
#    (leftJoin_ac_in_scoms['Cancel-Date'].isna())
# ]
# chk_conditions = leftJoin_ac_in_scoms[~(np.logical_and.reduce(conditions1))]
chk_conditions = leftJoin_ac_in_scoms_df[(np.logical_and.reduce(conditions1))]
chk_conditions.count()

circuit_id                 0
install_date               0
inactive_date              0
package                    0
package_startdate          0
package_enddate            0
ba_id                      0
customer_type_name         0
acc_type_update            0
org_dep_abbr               0
package_group_name_th      0
package_info_name          0
start_date_sale_package    0
end_date_sale_package      0
package_price              0
active_status              0
claims                     0
tenure_month               0
ipphone                    0
tv_box                     0
WiFi                       0
pay_first                  0
mobile                     0
dl_speed                   0
ul_speed                   0
dtype: int64

In [296]:
# สร้าง Function เพื่อตรวจสอบข้อมูลใน DataFrame

def missing_values_table(df):
        mis_val = df.isnull().sum()

        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)

        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})

        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)

        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")

        return mis_val_table_ren_columns

def data_explore(dataframe):
    missing_data = missing_values_table(leftJoin_ac_in_scoms_df)
    print("DATA EXPLORATION")
    print('*'*80)
    print("Shape of dataset : ",dataframe.shape)
    print('*'*80)
    print(dataframe.info())
    print('*'*80)
    print("STATISTICAL ANALYSIS OF NUMERICAL DATA")
    print('*'*80)
    print(dataframe.describe().T)
    print('*'*80)
    print("STATISTICAL ANALYSIS OF CATEGORICAL DATA")
    print('*'*80)
    print(dataframe.describe(exclude = ['float', 'int64']).T)
    print('*'*80)
    print("MISSING VALUES")
    print('*'*80)
    # print(dataframe.isna().sum().sort_values(ascending=False))
    print(missing_data)
    print('*'*80)
    print("MISSING VALUES IN %")
    print('*'*80)
    print(round(100* (dataframe.isnull().sum() / len(dataframe)).sort_values(ascending=False),2))
    print('*'*80)

In [297]:
missing_data = missing_values_table(leftJoin_ac_in_scoms_df)

Your selected dataframe has 25 columns.
There are 8 columns that have missing values.


In [298]:
data_explore(leftJoin_ac_in_scoms_df)


Your selected dataframe has 25 columns.
There are 8 columns that have missing values.
DATA EXPLORATION
********************************************************************************
Shape of dataset :  (159151, 25)
********************************************************************************
<class 'pandas.core.frame.DataFrame'>
Index: 159151 entries, 0 to 159999
Data columns (total 25 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   circuit_id               159151 non-null  object        
 1   install_date             159151 non-null  datetime64[ns]
 2   inactive_date            79585 non-null   datetime64[ns]
 3   package                  159151 non-null  object        
 4   package_startdate        159151 non-null  object        
 5   package_enddate          68 non-null      object        
 6   ba_id                    159151 non-null  object        
 7   customer_type_name       158412 n

In [309]:
billing_df = pd.read_csv('DataSet/SUMMARY_BILLING_2023.csv')

In [319]:
billing_df['ACCOUNT_TYPE'].value_counts()

ACCOUNT_TYPE
Residential - Normal                   43244
Business - SME (CSC/YorTor)            15861
Corporate - Key Account (JorKorBor)     3923
Business - Key Account (JorKorBor)       628
Corporate (CSC/YorTor)                   454
Residential - Foreigner                  168
NPO                                      158
High Ranking Govt Official               142
Residential - TOT Staff                  114
Government                               102
State Enterprise                          17
Residential - Monk                         9
SBU                                        1
Name: count, dtype: int64

In [318]:
# Define a list of desired promotions
desired_product = ['IP Phone', 'Fiber 2U']

# Filter rows where 'Promotion' is in the desired_promotions list
# filtered_data = billing_df[billing_df['PRODUCT DESC'].isin(desired_product)] 
# filtered_data
multiple_conditions = (billing_df['PRODUCT DESC'].isin(desired_product)) & (billing_df['TEL_NO'].str.contains('J', case=False))
filtered_data = billing_df[multiple_conditions]
filtered_data

,ACCOUNT_NO,TEL_NO,PRODUCT DESC,ACCOUNT_TYPE,BANK_CODE,CUSTOMER_REF,SERVICE_LOCATION NAME,INACTIVE_DATE,ACTIVE_DATE,AMOUNT_DUE
5,100007231,2246J9090,Fiber 2U,Residential - Normal,D,1-YB-690,ศูนย์บริการ NT อโศกดินแดง,15112566,26112563,20323.0
6,100008300,2246J7204,Fiber 2U,Residential - Normal,D,1-YB-807,ศูนย์บริการ NT อโศกดินแดง,15112566,26112563,20323.0
14,100025079,2252J1720,Fiber 2U,Residential - Normal,D,1-YE-378,ศูนย์บริการ NT สุขุมวิท 1,19112566,3092566,0.0
17,100036303,2251J2527,Fiber 2U,Corporate - Key Account (JorKorBor),D,1-1B46-73,ศูนย์บริการ NT สุขุมวิท 1,4082566,18122564,0.0
113,100080085,2231J1570,Fiber 2U,Residential - Normal,D,1-YN-1312,ศูนย์บริการ NT สุรวงศ์,1052566,27102565,9030.0
...,...,...,...,...,...,...,...,...,...,...
64812,104533366267,2373J4461,Fiber 2U,Residential - TOT Staff,D,CA-660901-0000304,ศูนย์บริการ NT รามคำแหง,11092566,26082566,15903.0
64813,104533366267,2373J4461,Fiber 2U,Residential - TOT Staff,D,CA-660901-0000304,ศูนย์บริการ NT รามคำแหง,25092566,12092566,27097.0
64814,104533372796,7553J8286,Fiber 2U,Residential - Normal,D,CA-660920-0000477,ศูนย์บริการ NT สิชล,29112566,21092566,20800.0
64817,104533378421,4357J9681,Fiber 2U,Residential - Normal,D,CA-660929-0000231,ศูนย์บริการ NT โพนทอง,20112566,6092566,14500.0
